This simulates a stock's price using Geometric Brownian Motion, and then prices an Asian and Barrier option using Monte Carlo simulations with varying path lengths.

In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
def simulate_stock_prices(S0, r, sigma, T, dt, paths):
    """Simulate stock prices using Geometric Brownian Motion."""
    
    steps = int(T / dt)
    dt = T / steps
    prices = np.zeros((steps + 1, paths))
    prices[0] = S0
    for t in range(1, steps + 1):
        z = np.random.standard_normal(paths)
        prices[t] = prices[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * z)
    return prices


In [3]:
def price_asian_option(S, K, r, T, option_type='call'):
    """Price an Asian option using Monte Carlo simulation."""

    avg_prices = np.mean(S, axis=0)
    if option_type == 'call': # Estimated option price
        payoffs = np.exp(-r * T) * np.maximum(avg_prices - K, 0)
    else:
        payoffs = np.exp(-r * T) * np.maximum(K - avg_prices, 0)
    return np.mean(payoffs)

def price_barrier_option(S, K, r, T, B, option_type='call', barrier_type='up-and-out'):
    """Price a Barrier option using Monte Carlo simulation."""

    if barrier_type == 'up-and-out':
        knocked_out = np.max(S, axis=0) > B
    else:  # 'down-and-out'
        knocked_out = np.min(S, axis=0) < B
    
    final_prices = S[-1]
    if option_type == 'call': # Estimated option price
        payoffs = np.where(knocked_out, 0, np.exp(-r * T) * np.maximum(final_prices - K, 0))
    else:
        payoffs = np.where(knocked_out, 0, np.exp(-r * T) * np.maximum(K - final_prices, 0))
    
    return np.mean(payoffs)


In [11]:
# Define parameters
S0 = 100  # Initial stock price
K = 105  # Strike price
B = 110  # Barrier level
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
T = 1  # Time to maturity (in years)
dt = 1/252  # Daily time step
paths_list = [1000, 5000, 10000]  # Different numbers of paths to simulate

for paths in paths_list:
    start_time = time.time()
    S = simulate_stock_prices(S0, r, sigma, T, dt, paths)
    asian_price = price_asian_option(S, K, r, T)
    barrier_price = price_barrier_option(S, K, r, T, B)
    end_time = time.time()
    
    print(f"Paths: {paths}, Asian Option Price: {asian_price:.4f}, Barrier Option Price: {barrier_price:.4f}, Time: {end_time - start_time:.2f} seconds")


Paths: 1000, Asian Option Price: 3.5739, Barrier Option Price: 0.0192, Time: 0.02 seconds
Paths: 5000, Asian Option Price: 3.5153, Barrier Option Price: 0.0253, Time: 0.05 seconds
Paths: 10000, Asian Option Price: 3.5004, Barrier Option Price: 0.0183, Time: 0.11 seconds
